--- Day 7: Camel Cards ---
Your all-expenses-paid trip turns out to be a one-way, five-minute ride in an airship. (At least it's a cool airship!) It drops you off at the edge of a vast desert and descends back to Island Island.

"Did you bring the parts?"

You turn around to see an Elf completely covered in white clothing, wearing goggles, and riding a large camel.

"Did you bring the parts?" she asks again, louder this time. You aren't sure what parts she's looking for; you're here to figure out why the sand stopped.

"The parts! For the sand, yes! Come with me; I will show you." She beckons you onto the camel.

After riding a bit across the sands of Desert Island, you can see what look like very large rocks covering half of the horizon. The Elf explains that the rocks are all along the part of Desert Island that is directly above Island Island, making it hard to even get there. Normally, they use big machines to move the rocks and filter the sand, but the machines have broken down because Desert Island recently stopped receiving the parts they need to fix the machines.

You've already assumed it'll be your job to figure out why the parts stopped when she asks if you can help. You agree automatically.

Because the journey will take a few days, she offers to teach you the game of Camel Cards. Camel Cards is sort of similar to poker except it's designed to be easier to play while riding a camel.

In Camel Cards, you get a list of hands, and your goal is to order them based on the strength of each hand. A hand consists of five cards labeled one of A, K, Q, J, T, 9, 8, 7, 6, 5, 4, 3, or 2. The relative strength of each card follows this order, where A is the highest and 2 is the lowest.

Every hand is exactly one type. From strongest to weakest, they are:

Five of a kind, where all five cards have the same label: AAAAA
Four of a kind, where four cards have the same label and one card has a different label: AA8AA
Full house, where three cards have the same label, and the remaining two cards share a different label: 23332
Three of a kind, where three cards have the same label, and the remaining two cards are each different from any other card in the hand: TTT98
Two pair, where two cards share one label, two other cards share a second label, and the remaining card has a third label: 23432
One pair, where two cards share one label, and the other three cards have a different label from the pair and each other: A23A4
High card, where all cards' labels are distinct: 23456
Hands are primarily ordered based on type; for example, every full house is stronger than any three of a kind.

If two hands have the same type, a second ordering rule takes effect. Start by comparing the first card in each hand. If these cards are different, the hand with the stronger first card is considered stronger. If the first card in each hand have the same label, however, then move on to considering the second card in each hand. If they differ, the hand with the higher second card wins; otherwise, continue with the third card in each hand, then the fourth, then the fifth.

So, 33332 and 2AAAA are both four of a kind hands, but 33332 is stronger because its first card is stronger. Similarly, 77888 and 77788 are both a full house, but 77888 is stronger because its third card is stronger (and both hands have the same first and second card).

To play Camel Cards, you are given a list of hands and their corresponding bid (your puzzle input). For example:

32T3K 765
T55J5 684
KK677 28
KTJJT 220
QQQJA 483
This example shows five hands; each hand is followed by its bid amount. Each hand wins an amount equal to its bid multiplied by its rank, where the weakest hand gets rank 1, the second-weakest hand gets rank 2, and so on up to the strongest hand. Because there are five hands in this example, the strongest hand will have rank 5 and its bid will be multiplied by 5.

So, the first step is to put the hands in order of strength:

32T3K is the only one pair and the other hands are all a stronger type, so it gets rank 1.
KK677 and KTJJT are both two pair. Their first cards both have the same label, but the second card of KK677 is stronger (K vs T), so KTJJT gets rank 2 and KK677 gets rank 3.
T55J5 and QQQJA are both three of a kind. QQQJA has a stronger first card, so it gets rank 5 and T55J5 gets rank 4.
Now, you can determine the total winnings of this set of hands by adding up the result of multiplying each hand's bid with its rank (765 * 1 + 220 * 2 + 28 * 3 + 684 * 4 + 483 * 5). So the total winnings in this example are 6440.

Find the rank of every hand in your set. What are the total winnings?

In [1]:
test_input = """
32T3K 765
T55J5 684
KK677 28
KTJJT 220
QQQJA 483
"""

In [2]:
from dataclasses import dataclass
from __future__ import annotations
from enum import Enum


class HandType(Enum):
    FIVE_OF_A_KIND = "07"
    FOUR_OF_A_KIND = "06"
    FULL_HOUSE = "05"
    THREE_OF_A_KIND = "04"
    TWO_PAIR = "03"
    ONE_PAIR = "02"
    HIGH_CARD = "01"


CARD_STR_TO_VALUE_MAP: dict[str, str] = {
    "A": "14",
    "K": "13",
    "Q": "12",
    "J": "11",
    "T": "10",
    "9": "09",
    "8": "08",
    "7": "07",
    "6": "06",
    "5": "05",
    "4": "04",
    "3": "03",
    "2": "02",
}


@dataclass
class Hand:
    card_value_list: list[str]
    bid: int
    hand_type: HandType
    hand_type_cards: list[str]


    @classmethod
    def from_str(cls, hand_str: str, j_for_joker: bool = False) -> Hand:
        card_str, bid_str = hand_str.strip().split()
        card_value_list = [CARD_STR_TO_VALUE_MAP[c] for c in card_str]
        if j_for_joker:
            card_value_list = ["01" if c == "11" else c for c in card_value_list]

        card_counter: dict[str, int] = {}
        for card_value in card_value_list:
            if card_value in card_counter:
                card_counter[card_value] += 1
            else:
                card_counter[card_value] = 1

        # remove jokers from consideration for types first
        n_jokers = card_counter.pop("01", 0)

        # create a reverse of count to card values map
        count_card_value_map: dict[int, list[str]] = {}
        for card_value, card_count in card_counter.items():
            if card_count in count_card_value_map:
                count_card_value_map[card_count].append(card_value)
            else:
                count_card_value_map[card_count] = [card_value]

        # hand specific cards are at most two values
        hand_type_cards = ["00", "00"]
        if 5 in count_card_value_map:
            hand_type = HandType.FIVE_OF_A_KIND
            hand_type_cards[0] = count_card_value_map[5][0]
        elif 4 in count_card_value_map:
            hand_type = HandType.FOUR_OF_A_KIND
            hand_type_cards[0] = count_card_value_map[4][0]
        elif 3 in count_card_value_map:
            # check for full house
            if 2 in count_card_value_map:
                hand_type = HandType.FULL_HOUSE
                hand_type_cards[0] = count_card_value_map[3][0]
                hand_type_cards[1] = count_card_value_map[2][0]
            else:
                hand_type = HandType.THREE_OF_A_KIND
                hand_type_cards[0] = count_card_value_map[3][0]
        elif 2 in count_card_value_map:
            # check for two pair
            if len(count_card_value_map[2]) == 2:
                hand_type = HandType.TWO_PAIR
                hand_type_cards[0] = count_card_value_map[2][0]
                hand_type_cards[1] = count_card_value_map[2][1]
            else:
                hand_type = HandType.ONE_PAIR
                hand_type_cards[0] = count_card_value_map[2][0]
        else:
            hand_type = HandType.HIGH_CARD

        for _ in range(n_jokers):
            # increase hand type for each jokers
            if hand_type == HandType.HIGH_CARD:
                hand_type =  HandType.ONE_PAIR
            elif hand_type == HandType.ONE_PAIR:
                hand_type =  HandType.THREE_OF_A_KIND
            elif hand_type == HandType.TWO_PAIR:
                hand_type =  HandType.FULL_HOUSE
            elif hand_type == HandType.THREE_OF_A_KIND:
                hand_type =  HandType.FOUR_OF_A_KIND
            elif hand_type == HandType.FULL_HOUSE:
                hand_type =  HandType.FOUR_OF_A_KIND
            elif hand_type == HandType.FOUR_OF_A_KIND:
                hand_type =  HandType.FIVE_OF_A_KIND

        return cls(
            card_value_list=card_value_list,
            bid=int(bid_str),
            hand_type=hand_type,
            hand_type_cards=hand_type_cards,
            )

    @property
    def sort_str(self) -> str:
        joined_hand_cards = "".join(self.card_value_list)
        sort_str = self.hand_type.value + joined_hand_cards
        return sort_str



@dataclass
class CamelGame:
    hand_list: list[Hand]

    @classmethod
    def from_str(cls, input_str: str, j_for_joker: bool = False) -> CamelGame:
        hand_list = [Hand.from_str(s, j_for_joker) for s in input_str.strip().split("\n")]
        return cls(hand_list=hand_list)

    @property
    def total_winnings(self) -> int:
        sorted_hands_low_first = sorted(self.hand_list, key=lambda x: int(x.sort_str))
        return sum([ (i + 1) * h.bid for i, h in enumerate(sorted_hands_low_first)])




In [3]:
test_game = CamelGame.from_str(test_input)

In [4]:
test_game.hand_list

[Hand(card_value_list=['03', '02', '10', '03', '13'], bid=765, hand_type=<HandType.ONE_PAIR: '02'>, hand_type_cards=['03', '00']),
 Hand(card_value_list=['10', '05', '05', '11', '05'], bid=684, hand_type=<HandType.THREE_OF_A_KIND: '04'>, hand_type_cards=['05', '00']),
 Hand(card_value_list=['13', '13', '06', '07', '07'], bid=28, hand_type=<HandType.TWO_PAIR: '03'>, hand_type_cards=['13', '07']),
 Hand(card_value_list=['13', '10', '11', '11', '10'], bid=220, hand_type=<HandType.TWO_PAIR: '03'>, hand_type_cards=['10', '11']),
 Hand(card_value_list=['12', '12', '12', '11', '14'], bid=483, hand_type=<HandType.THREE_OF_A_KIND: '04'>, hand_type_cards=['12', '00'])]

In [5]:
assert test_game.total_winnings == 6440

In [6]:
with open("../inputs/day7_input.txt") as f:
    input_str = f.read()

In [7]:
camel_game =  CamelGame.from_str(input_str)

In [8]:
camel_game.total_winnings

246409899

# Part 2

--- Part Two ---
To make things a little more interesting, the Elf introduces one additional rule. Now, J cards are jokers - wildcards that can act like whatever card would make the hand the strongest type possible.

To balance this, J cards are now the weakest individual cards, weaker even than 2. The other cards stay in the same order: A, K, Q, T, 9, 8, 7, 6, 5, 4, 3, 2, J.

J cards can pretend to be whatever card is best for the purpose of determining hand type; for example, QJJQ2 is now considered four of a kind. However, for the purpose of breaking ties between two hands of the same type, J is always treated as J, not the card it's pretending to be: JKKK2 is weaker than QQQQ2 because J is weaker than Q.

Now, the above example goes very differently:

32T3K 765
T55J5 684
KK677 28
KTJJT 220
QQQJA 483

32T3K is still the only one pair; it doesn't contain any jokers, so its strength doesn't increase.
KK677 is now the only two pair, making it the second-weakest hand.
T55J5, KTJJT, and QQQJA are now all four of a kind! T55J5 gets rank 3, QQQJA gets rank 4, and KTJJT gets rank 5.
With the new joker rule, the total winnings in this example are 5905.

Using the new joker rule, find the rank of every hand in your set. What are the new total winnings?



In [9]:
test_game_jokers = CamelGame.from_str(test_input, j_for_joker=True)

In [10]:
assert test_game_jokers.total_winnings == 5905

In [11]:
camel_game_jokers = CamelGame.from_str(input_str, j_for_joker=True)

In [12]:
camel_game_jokers.total_winnings

244848487